# Exploration des capacités GeoDjango avec les données GPS T-Drive

Ce notebook explore les fonctionnalités spatiales de GeoDjango en utilisant le dataset T-Drive de trajectoires de taxis.

**Objectifs:**
- Charger et structurer des données GPS
- Créer des objets géométriques GeoDjango
- Effectuer des opérations spatiales
- Analyser et visualiser les trajectoires

## 1. Configuration et Imports

In [3]:
import os
import sys
from pathlib import Path
import django
from django.conf import settings

# Only set up Django if not already configured
if not settings.configured:
    BASE_DIR = Path.cwd().parent  # server/
    sys.path.append(str(BASE_DIR))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
    django.setup()

print("✓ Django configuré avec succès")


✓ Django configuré avec succès


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from django.contrib.gis.geos import Point, LineString, Polygon, MultiPoint
from django.contrib.gis.measure import D
from django.contrib.gis.db.models.functions import Distance, Area, Length
from django.db.models import Count, Avg, Max, Min, F, Sum
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration des graphiques
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliothèques importées")

/home/paulh/TAI/.venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


✓ Bibliothèques importées


In [5]:
# Import des modèles Django (à adapter selon votre structure)
# from apps.mobility.models import GPSTrace, Trip
# from apps.poi.models import POI

print("✓ Modèles importés (décommenter selon vos modèles)")

✓ Modèles importés (décommenter selon vos modèles)


## 2. Chargement des Données T-Drive

In [15]:
import pandas as pd
from pathlib import Path

def load_tdi_trajectory_data_from_folder(data_dir):
    """
    Load T-Drive trajectory data from a local folder containing TXT/CSV files.
    
    Args:
        data_dir (str or Path): Path to the folder containing trajectory files.
    
    Returns:
        pd.DataFrame
    """
    data_dir = Path(data_dir)
    
    # Find all TXT or CSV files in the folder
    trajectory_files = list(data_dir.glob("**/*.txt")) + list(data_dir.glob("**/*.csv"))
    if not trajectory_files:
        raise RuntimeError(f"No trajectory files found in {data_dir}")
    
    dfs = []
    for traj_file in trajectory_files:
        print(f"📊 Loading trajectory data from {traj_file}")
        df = pd.read_csv(traj_file, delimiter=',', header=None,
                         names=['taxi_id', 'timestamp', 'longitude', 'latitude'])
        dfs.append(df)
    
    # Combine all files
    df_all = pd.concat(dfs, ignore_index=True)
    
    # Convert types
    df_all['longitude'] = pd.to_numeric(df_all['longitude'], errors='coerce')
    df_all['latitude'] = pd.to_numeric(df_all['latitude'], errors='coerce')
    df_all['timestamp'] = pd.to_datetime(df_all['timestamp'], errors='coerce')
    df_all = df_all.dropna()
    df_all = df_all.sort_values(['taxi_id', 'timestamp']).reset_index(drop=True)
    
    print(f"✅ Loaded {len(df_all)} GPS points from {df_all['taxi_id'].nunique()} taxis")
    return df_all

# Path to the local T-Drive folder
tdrive_folder = BASE_DIR / "data" / "tdrive"
df_gps = load_tdi_trajectory_data_from_folder(tdrive_folder)


📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/1.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/100.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/1000.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10000.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10001.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10002.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10003.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10004.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10005.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10006.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10007.txt
📊 Loading trajectory data from /home/paulh/TAI/server/data/tdrive/10008.txt
📊 Loading trajectory d

/tmp/ipykernel_10248/1520422667.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(dfs, ignore_index=True)


✅ Loaded 17662984 GPS points from 10336 taxis


In [16]:
# Statistiques descriptives
if df_gps.empty:
    print("⚠️ Le DataFrame est vide, impossible de générer des statistiques.")
else:
    print("Informations sur le dataset:")
    print(df_gps.info())
    print("\nStatistiques numériques:")
    display(df_gps.describe())
    
    print(f"\nAperçu des 10 premiers points GPS:")
    display(df_gps.head(10))


Informations sur le dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17662984 entries, 0 to 17662983
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   taxi_id    object        
 1   timestamp  datetime64[ns]
 2   longitude  float64       
 3   latitude   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 539.0+ MB
None

Statistiques numériques:


,timestamp,longitude,latitude
count,17662984,1.766298e+07,1.766298e+07
mean,2008-02-05 11:41:36.273897216,1.161801e+02,3.984973e+01
min,2008-02-02 13:30:44,0.000000e+00,0.000000e+00
25%,2008-02-03 22:21:00,1.163340e+02,3.987953e+01
50%,2008-02-05 10:43:15,1.164065e+02,3.992271e+01
75%,2008-02-06 22:11:56,1.164723e+02,3.998121e+01
max,2008-02-08 17:39:19,2.553000e+02,9.606767e+01
std,NaN,5.440849e+00,1.904082e+00



Aperçu des 10 premiers points GPS:


,taxi_id,timestamp,longitude,latitude
0,1,2008-02-02 15:36:08,116.51172,39.92123
1,1,2008-02-02 15:46:08,116.51135,39.93883
2,1,2008-02-02 15:46:08,116.51135,39.93883
3,1,2008-02-02 15:56:08,116.51627,39.91034
4,1,2008-02-02 16:06:08,116.47186,39.91248
5,1,2008-02-02 16:16:08,116.47217,39.92498
6,1,2008-02-02 16:26:08,116.47179,39.90718
7,1,2008-02-02 16:36:08,116.45617,39.90531
8,1,2008-02-02 17:00:24,116.47191,39.90577
9,1,2008-02-02 17:10:24,116.50661,39.91450


## 3. Création d'Objets Géométriques GeoDjango

### 3.1 Points GPS

In [18]:
# Création de Points GeoDjango (SRID 4326 = WGS84)
df_gps['point'] = df_gps.apply(
    lambda row: Point(row['longitude'], row['latitude'], srid=4326),
    axis=1
)

print(f"✓ {len(df_gps)} Points créés (SRID 4326 - WGS84)")
print(f"\nExemple de Point:")
print(df_gps['point'].iloc[0])
print(f"Type: {df_gps['point'].iloc[0].geom_type}")
print(f"Coordonnées: {df_gps['point'].iloc[0].coords}")

✓ 17662984 Points créés (SRID 4326 - WGS84)

Exemple de Point:
SRID=4326;POINT (116.51172 39.92123)
Type: Point
Coordonnées: (116.51172, 39.92123)


### 3.2 LineStrings (Trajectoires)

In [19]:
# Création de LineString par trajectoire de taxi
trajectories = {}
for taxi_id in df_gps['taxi_id'].unique()[:5]:  # Premiers 5 taxis
    taxi_data = df_gps[df_gps['taxi_id'] == taxi_id].sort_values('timestamp')
    coords = [(row['longitude'], row['latitude']) for _, row in taxi_data.iterrows()]
    
    if len(coords) >= 2:
        trajectories[taxi_id] = LineString(coords, srid=4326)
        
print(f"✓ {len(trajectories)} trajectoires LineString créées")
print(f"\nExemple de LineString (Taxi {list(trajectories.keys())[0]}):")
first_traj = list(trajectories.values())[0]
print(f"Type: {first_traj.geom_type}")
print(f"Nombre de points: {len(first_traj.coords)}")
print(f"Premier point: {first_traj.coords[0]}")
print(f"Dernier point: {first_traj.coords[-1]}")

✓ 5 trajectoires LineString créées

Exemple de LineString (Taxi 1):
Type: LineString
Nombre de points: 588
Premier point: (116.51172, 39.92123)
Dernier point: (116.54723, 39.90841)


### 3.3 Polygon (Zone d'étude)

In [20]:
# Création d'un Polygon - Bounding box autour des données
min_lon, max_lon = df_gps['longitude'].min(), df_gps['longitude'].max()
min_lat, max_lat = df_gps['latitude'].min(), df_gps['latitude'].max()

study_area = Polygon((
    (min_lon, min_lat),
    (max_lon, min_lat),
    (max_lon, max_lat),
    (min_lon, max_lat),
    (min_lon, min_lat)
), srid=4326)

print(f"✓ Zone d'étude créée: {study_area.geom_type}")
print(f"  Bounds: [{min_lon:.4f}, {min_lat:.4f}] à [{max_lon:.4f}, {max_lat:.4f}]")
print(f"  Nombre de sommets: {len(study_area.coords[0])}")
print(f"  SRID: {study_area.srid}")

✓ Zone d'étude créée: Polygon
  Bounds: [0.0000, 0.0000] à [255.3000, 96.0677]
  Nombre de sommets: 5
  SRID: 4326


## 4. Opérations Spatiales GeoDjango

### 4.1 Calcul de Distances

In [21]:
sample_points = df_gps.head(10)['point'].tolist()

if len(sample_points) >= 2:
    # Distance en degrés (système WGS84)
    dist_degrees = sample_points[0].distance(sample_points[1])
    print(f"Distance entre 2 premiers points:")
    print(f"  En degrés: {dist_degrees:.6f}°")
    
    # Transformation en projection métrique pour distance réelle
    p1_metric = sample_points[0].transform(3857, clone=True)  # Web Mercator
    p2_metric = sample_points[1].transform(3857, clone=True)
    dist_meters = p1_metric.distance(p2_metric)
    print(f"  En mètres: {dist_meters:.2f} m")
    print(f"  En kilomètres: {dist_meters/1000:.3f} km")

Distance entre 2 premiers points:
  En degrés: 0.017604°
  En mètres: 2555.30 m
  En kilomètres: 2.555 km


### 4.2 Longueur de Trajectoires

In [22]:
print("Longueur des trajectoires:")
print("-" * 50)

for taxi_id, trajectory in list(trajectories.items())[:3]:
    # Longueur en degrés
    length_deg = trajectory.length
    
    # Transformation en projection métrique
    traj_metric = trajectory.transform(3857, clone=True)
    length_m = traj_metric.length
    length_km = length_m / 1000
    
    print(f"Taxi {taxi_id}:")
    print(f"  Longueur: {length_km:.2f} km")
    print(f"  Nombre de points: {len(trajectory.coords)}")
    print()

Longueur des trajectoires:
--------------------------------------------------
Taxi 1:
  Longueur: 575.40 km
  Nombre de points: 588

Taxi 2:
  Longueur: 2181.24 km
  Nombre de points: 1674

Taxi 3:
  Longueur: 1742.64 km
  Nombre de points: 1371



### 4.3 Tests de Contenance (Contains/Within)

In [23]:
# Test si les points sont dans la zone d'étude
points_in_area = sum(1 for point in sample_points if study_area.contains(point))
print(f"Points dans la zone d'étude: {points_in_area}/{len(sample_points)}")

# Test inverse: le point est-il dans le polygone?
test_point = sample_points[0]
print(f"\nLe point {test_point.coords} est-il dans la zone?")
print(f"  study_area.contains(point): {study_area.contains(test_point)}")
print(f"  point.within(study_area): {test_point.within(study_area)}")

Points dans la zone d'étude: 10/10

Le point (116.51172, 39.92123) est-il dans la zone?
  study_area.contains(point): True
  point.within(study_area): True


### 4.4 Buffer (Zones Tampons)

In [24]:
# Création d'un buffer autour du point central
center_point = Point(
    df_gps['longitude'].mean(), 
    df_gps['latitude'].mean(), 
    srid=4326
)

# Buffer de ~500m (approximatif en degrés)
buffer_500m = center_point.buffer(0.005)

print(f"Point central: {center_point.coords}")
print(f"\nBuffer créé:")
print(f"  Type: {buffer_500m.geom_type}")
print(f"  Rayon: ~500m")
print(f"  Nombre de sommets: {len(buffer_500m.coords[0])}")

Point central: (116.1801431941528, 39.849730161835055)

Buffer créé:
  Type: Polygon
  Rayon: ~500m
  Nombre de sommets: 33


### 4.5 Intersections Spatiales

In [25]:
# Trajectoires intersectant le buffer
print("Analyse des intersections:")
print("-" * 50)

intersecting_trajectories = 0
for taxi_id, traj in trajectories.items():
    if traj.intersects(buffer_500m):
        intersecting_trajectories += 1
        print(f"✓ Taxi {taxi_id} traverse la zone centrale")
    else:
        print(f"✗ Taxi {taxi_id} ne traverse pas la zone centrale")

print(f"\nTotal: {intersecting_trajectories}/{len(trajectories)} trajectoires intersectant le buffer")

Analyse des intersections:
--------------------------------------------------
✗ Taxi 1 ne traverse pas la zone centrale
✗ Taxi 2 ne traverse pas la zone centrale
✗ Taxi 3 ne traverse pas la zone centrale
✗ Taxi 4 ne traverse pas la zone centrale
✗ Taxi 5 ne traverse pas la zone centrale

Total: 0/5 trajectoires intersectant le buffer


## 5. Exemples de Requêtes Django ORM Spatiales

Ces exemples montrent comment utiliser GeoDjango ORM pour des requêtes spatiales avancées dans votre application Django.

In [26]:
# EXEMPLE 1: Trouver tous les points GPS dans un rayon
print("""# Requête 1: Points GPS à proximité
center = Point(longitude, latitude, srid=4326)
nearby_traces = GPSTrace.objects.filter(
    location__distance_lte=(center, D(km=1))
).annotate(
    distance=Distance('location', center)
).order_by('distance')[:10]
""")

# Requête 1: Points GPS à proximité
center = Point(longitude, latitude, srid=4326)
nearby_traces = GPSTrace.objects.filter(
    location__distance_lte=(center, D(km=1))
).annotate(
    distance=Distance('location', center)
).order_by('distance')[:10]



In [27]:
# EXEMPLE 2: Trajectoires intersectant une zone
print("""# Requête 2: Trajectoires dans une zone
zone = Polygon(coordinates, srid=4326)
trips_in_zone = Trip.objects.filter(
    trajectory__intersects=zone
).count()
""")

# Requête 2: Trajectoires dans une zone
zone = Polygon(coordinates, srid=4326)
trips_in_zone = Trip.objects.filter(
    trajectory__intersects=zone
).count()



In [28]:
# EXEMPLE 3: POIs à proximité d'une trajectoire
print("""# Requête 3: POIs proches d'une trajectoire
trajectory = LineString(coords, srid=4326)
nearby_pois = POI.objects.filter(
    location__distance_lte=(trajectory, D(m=500))
).annotate(
    distance=Distance('location', trajectory)
).order_by('distance')
""")

# Requête 3: POIs proches d'une trajectoire
trajectory = LineString(coords, srid=4326)
nearby_pois = POI.objects.filter(
    location__distance_lte=(trajectory, D(m=500))
).annotate(
    distance=Distance('location', trajectory)
).order_by('distance')



In [29]:
# EXEMPLE 4: Longueur totale par véhicule
print("""# Requête 4: Statistiques de distance par véhicule
from django.contrib.gis.db.models.functions import Length

trip_stats = Trip.objects.annotate(
    length_km=Length('trajectory') / 1000
).values('vehicle_id').annotate(
    total_km=Sum('length_km'),
    avg_km=Avg('length_km'),
    trip_count=Count('id')
).order_by('-total_km')
""")

# Requête 4: Statistiques de distance par véhicule
from django.contrib.gis.db.models.functions import Length

trip_stats = Trip.objects.annotate(
    length_km=Length('trajectory') / 1000
).values('vehicle_id').annotate(
    total_km=Sum('length_km'),
    avg_km=Avg('length_km'),
    trip_count=Count('id')
).order_by('-total_km')



In [30]:
# EXEMPLE 5: Bounding Box Query
print("""# Requête 5: Points dans un rectangle
bbox = Polygon.from_bbox((min_lon, min_lat, max_lon, max_lat))
points_in_bbox = GPSTrace.objects.filter(
    location__within=bbox
).count()
""")

# Requête 5: Points dans un rectangle
bbox = Polygon.from_bbox((min_lon, min_lat, max_lon, max_lat))
points_in_bbox = GPSTrace.objects.filter(
    location__within=bbox
).count()



In [31]:
# EXEMPLE 6: Plus proche voisin
print("""# Requête 6: POI le plus proche pour chaque trace
from django.db.models import OuterRef, Subquery

nearest_poi = POI.objects.filter(
    location__distance_lte=(OuterRef('location'), D(m=100))
).order_by(
    Distance('location', OuterRef('location'))
)[:1]

traces_with_nearest_poi = GPSTrace.objects.annotate(
    nearest_poi_id=Subquery(nearest_poi.values('id')),
    nearest_poi_distance=Subquery(
        nearest_poi.annotate(
            dist=Distance('location', OuterRef('location'))
        ).values('dist')
    )
)
""")

# Requête 6: POI le plus proche pour chaque trace
from django.db.models import OuterRef, Subquery

nearest_poi = POI.objects.filter(
    location__distance_lte=(OuterRef('location'), D(m=100))
).order_by(
    Distance('location', OuterRef('location'))
)[:1]

traces_with_nearest_poi = GPSTrace.objects.annotate(
    nearest_poi_id=Subquery(nearest_poi.values('id')),
    nearest_poi_distance=Subquery(
        nearest_poi.annotate(
            dist=Distance('location', OuterRef('location'))
        ).values('dist')
    )
)



## 6. Analyse Statistique Spatiale

### 6.1 Calcul des Vitesses

In [32]:
# Calcul des vitesses entre points consécutifs
df_gps_sorted = df_gps.sort_values(['taxi_id', 'timestamp']).copy()
df_gps_sorted['time_diff'] = df_gps_sorted.groupby('taxi_id')['timestamp'].diff().dt.total_seconds()
df_gps_sorted['dist_diff'] = 0.0

print("Calcul des distances entre points consécutifs...")
for i in range(1, len(df_gps_sorted)):
    if df_gps_sorted.iloc[i]['taxi_id'] == df_gps_sorted.iloc[i-1]['taxi_id']:
        p1 = df_gps_sorted.iloc[i-1]['point'].transform(3857, clone=True)
        p2 = df_gps_sorted.iloc[i]['point'].transform(3857, clone=True)
        df_gps_sorted.iloc[i, df_gps_sorted.columns.get_loc('dist_diff')] = p1.distance(p2)

# Calcul des vitesses en km/h
df_gps_sorted['speed_kmh'] = (df_gps_sorted['dist_diff'] / df_gps_sorted['time_diff']) * 3.6
df_gps_sorted = df_gps_sorted[df_gps_sorted['speed_kmh'].notna()]
df_gps_sorted = df_gps_sorted[df_gps_sorted['speed_kmh'] < 200]  # Filtrage valeurs aberrantes

print("✓ Vitesses calculées")

Calcul des distances entre points consécutifs...


GDAL_ERROR 1: b'PROJ: webmerc: Invalid latitude'


GDALException: OGR failure.

In [ ]:
# Statistiques de vitesse
print("STATISTIQUES DE VITESSE")
print("=" * 50)
print(f"Vitesse moyenne: {df_gps_sorted['speed_kmh'].mean():.2f} km/h")
print(f"Vitesse médiane: {df_gps_sorted['speed_kmh'].median():.2f} km/h")
print(f"Vitesse min: {df_gps_sorted['speed_kmh'].min():.2f} km/h")
print(f"Vitesse max: {df_gps_sorted['speed_kmh'].max():.2f} km/h")
print(f"Écart-type: {df_gps_sorted['speed_kmh'].std():.2f} km/h")

# Percentiles
print("\nPercentiles:")
for p in [25, 50, 75, 90, 95]:
    print(f"  {p}e percentile: {df_gps_sorted['speed_kmh'].quantile(p/100):.2f} km/h")

### 6.2 Distribution Spatiale

In [ ]:
print("DISTRIBUTION SPATIALE")
print("=" * 50)
print(f"Longitude: [{min_lon:.6f}, {max_lon:.6f}]")
print(f"Latitude: [{min_lat:.6f}, {max_lat:.6f}]")
print(f"Étendue: {(max_lon - min_lon):.6f}° × {(max_lat - min_lat):.6f}°")
print(f"\nCentre approximatif:")
print(f"  Longitude: {df_gps['longitude'].mean():.6f}")
print(f"  Latitude: {df_gps['latitude'].mean():.6f}")

### 6.3 Distribution Temporelle